In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

memory = ConversationBufferMemory(
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = {"question": RunnablePassthrough(), "context": retriever, "history": load_memory} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)


In [2]:
invoke_chain("Is Aaronson guilty?")

Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [3]:
invoke_chain("What message did he write in the table?")

He wrote "2+2=5" in the dust on the table.


In [4]:
invoke_chain("Who is Julia?")

Julia is a character who was involved with the protagonist in the story.


In [5]:
invoke_chain("What questions did I asked?")

You asked about Aaronson's guilt and the message he wrote on the table.
